In [1]:
import pandas as pd
import requests
import gzip

In [12]:
panelapp_url = "https://panelapp.genomicsengland.co.uk/api/v1/panels/"
panel_list = requests.get(panelapp_url).json()["results"]

panelapp_ar_genes = set()
for panel in panel_list:
    panel_id = panel["id"]
    # print(f"Processing PanelApp panel {panel_id} - {panel['name']}")
    panel_detail_url = f"https://panelapp.genomicsengland.co.uk/api/v1/panels/{panel_id}/"
    detail = requests.get(panel_detail_url).json()
    for gene in detail.get("genes", []):
        # print(f"  Processing gene {gene['gene_data']['gene_symbol']}: {gene['mode_of_inheritance']}")
        if "mode_of_inheritance" in gene and "BIALLELIC" in gene["mode_of_inheritance"]:
            print(f"  Found biallelic gene {gene['gene_data']['gene_symbol']}")
            panelapp_ar_genes.add(gene["gene_data"]["gene_symbol"])

print(len(panelapp_ar_genes), "Autosomal recessive genes found in PanelApp")
panelapp_df = pd.DataFrame(sorted(panelapp_ar_genes), columns=["symbol"])
panelapp_df["source"] = "PanelApp"

  Found biallelic gene ACAD9
  Found biallelic gene ACADM
  Found biallelic gene ACADVL
  Found biallelic gene AGL
  Found biallelic gene ALDOA
  Found biallelic gene AMPD1
  Found biallelic gene ANO5
  Found biallelic gene CHKB
  Found biallelic gene COQ4
  Found biallelic gene COQ8A
  Found biallelic gene DGUOK
  Found biallelic gene DYSF
  Found biallelic gene ENO3
  Found biallelic gene ETFA
  Found biallelic gene ETFB
  Found biallelic gene ETFDH
  Found biallelic gene FDX2
  Found biallelic gene FKRP
  Found biallelic gene FLAD1
  Found biallelic gene GAA
  Found biallelic gene GBE1
  Found biallelic gene GMPPB
  Found biallelic gene GYG1
  Found biallelic gene GYS1
  Found biallelic gene HADHA
  Found biallelic gene HADHB
  Found biallelic gene LDHA
  Found biallelic gene LPIN1
  Found biallelic gene MLIP
  Found biallelic gene OBSCN
  Found biallelic gene PFKM
  Found biallelic gene PGAM2
  Found biallelic gene PGM1
  Found biallelic gene PYGM
  Found biallelic gene SGCA
  Foun